<h1>Ditchley S2DS project August 2020 - Code Pipeline<h1>
    <h2>Team: Adam Hawken, Luca Lamoni, Elizabeth Nicholson, Robert Webster<h2>

In [7]:
# Import modules and set up working directory
import sys
import os
import time
import logging
import json
import csv
import threading
import queue
import asyncio 
import nest_asyncio
nest_asyncio.apply()
import twint
import pandas as pd

sys.path.insert(1, 'C:/Users/Luca/Aug20_Ditchley')

<h3>Section 1: Getting journalist twitter handles according to a keyword<h3>
    <h4>The journalist scraping is performed at the web address https://www.journalism.co.uk/prof/?chunk=0&cmd=default<h4>

In [8]:
from src.data import journalists as journos
keyword = 'cybersecurity'
journo_handles = journos.get_handles_by_keyword(keyword)
print(len(journo_handles))

3


<h3>Section 2. Loop over selected journalists handles and download their tweets<h3>
    <h4>Section 2.1: Download tweets using Twint<h4> 

In [ ]:
from src.data import twint_tools as tt
# define keyword arguments
kwargs = {'date_range':('2020-08-01 00:00:00', None),
         'n_retries':5,
         'suppress':False}
# multi threading
tt.twint_in_queue(tt._search_tweets_by_user, 3, journo_handles, args=('../data/raw/'+keyword+'_',), kwargs=kwargs)

In [ ]:
# Joined all the individual csv into one dataframe
cyber_test = tt.join_tweet_csv(journo_handles, keyword)
# Check
cyber_test.head()

In [ ]:
# Standardise the twint output 
from src.data import data_cleanup as dc
#Create the standardized template
test_twint = dc.standard_tweet_dataset_setup('test_twint')
test_twint
#fill the template
standard_tweet_twint = dc.fill_standard_tweet_dataset_with_twint(test_twint, cyber_test)
# Check
standard_tweet_twint.head()

In [ ]:
# Save the dataframe
standard_tweet_twint.to_csv('../data/processed/' + keyword + '_standard_tweets_twint.csv',index=False)

<h4>Section 2.2: Extract mentions from Twint dataset<h4> 

In [ ]:
# from the twint dataset, extract mentions based on tweet id and save in a separate csv
mentions_twint  = dc.twint_mentions_to_df(cyber_test)
# Check
mentions_twint.head()

In [ ]:
# Save the dataframe
mentions_twint.to_csv('../data/processed/' + keyword + '_mentions_twint.csv',index=False)

<h4>Section 2.3: Scrape list of friends for each journalist<h4> 

In [ ]:
from src.data import twint_tools as tt
# define keyword arguments
kwargs = {#'date_range':('2020-08-01 00:00:00', None),
         'n_retries':5,
         'suppress':False}
# multi threading
tt.twint_in_queue(tt._get_friends, 3, journo_handles, args=('../data/raw/'+keyword+'_',), kwargs=kwargs)

In [ ]:
# Concatenate all the individual lists into one dataframe with journalist and its friends
friends_csv = tt.join_friends_csv(journo_handles,keyword) # this function has a bug, the first friend name is 'username'

In [ ]:
# Save the dataframe
friends_csv.to_csv('../data/raw/'+keyword+'_journalist_friends.csv', index=False)

<h4>Section 2.4: Scrape for each journalist the user information using Twitter API<h4> 

In [4]:
import json
import tweepy
from src.data import api_tweepy as api
#Load twitter API credentials and return a tweepy API instance
tw_api = api.connect_API('../src/data/twitter_credentials.json')

In [9]:
# Scrape user information using the API
from src.data import api_user_tools as api_tools
api_users = api_tools.batch_pull_user_info(tw_api,journo_handles)

TypeError: batch_pull_user_info() missing 1 required positional argument: 'kwargs'

<h4>Section 2b: Download tweets using API<h4>

In [ ]:
!pip uninstall tweepy

<h3>Section 3. Loop over selected journalists handles and download their lists of friends<h3>